In [1]:
import keras 
import keras_retinanet
from keras import layers
from keras import backend as b
from keras.utils.vis_utils import plot_model
from keras_retinanet import models
#-----------------------------------------------------------------------------
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Dense ,Concatenate,concatenate ,AveragePooling2D, Dropout ,Flatten,Dense,Input
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from keras.models import Model ,load_model
from tensorflow.keras.applications import  VGG16
#-----------------------------------------------------------------------------
import time
from timeit import default_timer as timer   
import cv2
import argparse
import os
import numpy as np
import matplotlib.pyplot as plt
import gc
from numba import jit, cuda
#-----------------------------------------------------------------------------
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths


from absl import app, flags, logging
from absl.flags import FLAGS

from tensorflow.python.saved_model import tag_constants
from PIL import Image


from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

import imutils
from imutils.video import VideoStream
import gc

from twilio.rest import Client
import sys

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
pwd

'C:\\Users\\mnaga\\Desktop\\Graduation_Project'

In [4]:
#Path for custom weapon folder

In [5]:
cd Custom_Weapon

C:\Users\mnaga\Desktop\Graduation_Project\Custom_Weapon


In [6]:
pwd

'C:\\Users\\mnaga\\Desktop\\Graduation_Project\\Custom_Weapon'

In [7]:
import core.utils as utils
from core.yolov4 import filter_boxes

In [8]:
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
input_size = 416
saved_model_loaded = tf.saved_model.load(r"./custom-416", tags=[tag_constants.SERVING])
infer = saved_model_loaded.signatures['serving_default']

In [9]:
def weapon (frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(frame)
    frame_size = frame.shape[:2]
    image_data = cv2.resize(frame, (input_size, input_size))
    image_data = image_data / 255.
    image_data = image_data[np.newaxis, ...].astype(np.float32)
    start_time = time.time()

    batch_data = tf.constant(image_data)
    pred_bbox = infer(batch_data)
    for key, value in pred_bbox.items():
        boxes = value[:, :, 0:4]
        pred_conf = value[:, :, 4:]
#----------------------------------------------------------
    boxes, scores, classes, valid_detections = tf.image.combined_non_max_suppression(
        boxes=tf.reshape(boxes, (tf.shape(boxes)[0], -1, 1, 4)),
        scores=tf.reshape( pred_conf, (tf.shape(pred_conf)[0], -1, tf.shape(pred_conf)[-1])),
        max_output_size_per_class=50,
        max_total_size=50,
        #threshold and score
        iou_threshold= 0.6,
        score_threshold= 0.6)
    pred_bbox = [boxes.numpy(), scores.numpy(), classes.numpy(), valid_detections.numpy()]
    image , cc = utils.draw_bbox(frame, pred_bbox)
    fps = 1.0 / (time.time() - start_time)
    result = np.asarray(image)
    result = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return cc , result

In [10]:
OPENCV_DNN_OPENCL_ALLOW_ALL_DEVICES= 1

In [11]:
def detect_and_predict_mask(frame, faceNet, maskNet):
	# grab the dimensions of the frame and then construct a blob
	# from it
	(hight, width) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(frame, 1.0, (224,224),
		(104.0, 177.0, 123.0))

	# pass the blob through the network and obtain the face detections
	faceNet.setInput(blob)
	detections = faceNet.forward()
	print(detections.shape)

	# initialize our list of faces, their corresponding locations,
	# and the list of predictions from our face mask network
	faces = []
	locs = []
	preds = []

	# loop over the detections
	for i in range(0, detections.shape[2]):
		# extract the confidence (i.e., probability) associated with
		# the detection
		confidence = detections[0, 0, i, 2]

		# filter out weak detections by ensuring the confidence is
		# greater than the minimum confidence
		if confidence > 0.5:
			# compute the (x, y)-coordinates of the bounding box for
			# the object
			box = detections[0, 0, i, 3:7] * np.array([width, hight, width, hight])
			(startX, startY, endX, endY) = box.astype("int")

			# ensure the bounding boxes fall within the dimensions of
			# the frame
			(startX, startY) = (max(0, startX), max(0, startY))
			(endX, endY) = (min(width - 1, endX), min(hight - 1, endY))

			# extract the face ROI, convert it from BGR to RGB channel
			# ordering, resize it to 224x224, and preprocess it
			face = frame[startY:endY, startX:endX]
			print(face.shape)
			face = cv2.cvtColor(face, cv2.COLOR_RGB2BGR)  #COLOR_RGB2GRAY
			face = cv2.resize(face, (224,224))
			face = img_to_array(face)
			face = preprocess_input(face)

			# add the face and bounding boxes to their respective
			# lists
			faces.append(face)
			locs.append((startX, startY, endX, endY))

	# only make a predictions if at least one face was detected
	if len(faces) > 0:
		# for faster inference we'll make batch predictions on *all*
		# faces at the same time rather than one-by-one predictions
		# in the above `for` loop
		faces = np.array(faces, dtype="float32")
		preds = maskNet.predict(faces, batch_size=64)

	# return a 2-tuple of the face locations and their corresponding
	# locations
	return (locs, preds)

In [12]:
def Shoplifting_prediction (image,features) :
    color = (0,0,255)
    RGB_img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    res = cv2.resize(RGB_img, dsize=(64, 64),interpolation=cv2.INTER_AREA)
    res = (res/255.).astype(np.float16)
    global label_shoplifting
    # Reshaping to put the number of the videos
    features.append(res)

    if len(features) == 30:
        AfterReshape = np.reshape(np.array(features), (1 ,30,64,64 ,3)).astype(np.float16)
        prediction = model_Shoplifting.predict(AfterReshape)
        if prediction[0][1] > 0.972: 
            print("Normal with :",prediction[0][1])
            color = (0,255,0)
            label_shoplifting = "Normal"
        else :
            print("Shoplifting :",prediction[0][0])
            color = (0,0,255)
            label_shoplifting = "Shoplifting"
        features = []
    cv2.putText(image,label_shoplifting, (0,30), cv2.FONT_HERSHEY_SIMPLEX, 0.8,color, 2, cv2.LINE_AA)
    return label_shoplifting

In [13]:
def mask(frame, faceNet, maskNet): 
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)
    label = ""
    if len(preds)==1 :
        (masked_face , mask, withoutMask) = preds[0]
        for (box, pred) in zip(locs, preds):

            # unpack the bounding box and predictions
            (startX, startY, endX, endY) = box

            if (mask > withoutMask and mask > masked_face):
                label = "Normal Mask" 
            elif (withoutMask > mask and withoutMask > masked_face):
                label = "No Mask" 
            elif (masked_face > mask and masked_face > withoutMask):
                label = "Threat"

            if label == "Normal Mask":
                color = (0, 255, 0) 
            elif label == "No Mask":
                color = (255, 0, 0)
            elif label == "Threat":
                color = (0, 0, 255)

            l = "{}: {:.2f}%".format(label, max(masked_face , mask, withoutMask) * 100)
            # frame
            cv2.putText(frame, l, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
            return label
    else: 
        return "No thing"

In [14]:
def Voilence_prediction (image,features) :
    color = (0,0,255)    
    RGB_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) 
    res = cv2.resize(RGB_img, dsize=(img_size, img_size),interpolation=cv2.INTER_AREA)
    global label_Voilence
    features.append(res)

    if len(features) == 40:
        test_data = np.array(features)
        resul = (test_data / 255.).astype(np.float16)
        transfer_values_testing2 = image_model_transfer.predict(resul)
        AfterReshape2 =  np.reshape(transfer_values_testing2, (1 , 40 , 4096))
 
        y_score = model_Voilence.predict(AfterReshape2)  
        li = [[item] for c_list in y_score for item in y_score]
        
        First = li[0][0][0]
        second = li[0][0][1]
    

        labels = ['Violence', 'Non-Violence']
        x = np.arange(len(labels))
        y = []

        if First > second:
            y = [First, 0]
            label_Voilence = "Violence" #+ str (y)
        else:
            y = [0, second]
            label_Voilence = "Non-Violence" #+ str (y)
          
        features = []
    #cv2.putText(image,label_Voilence, (0,60), cv2.FONT_HERSHEY_SIMPLEX, 0.8,color, 2, cv2.LINE_AA)
    cv2.putText(image,label_Voilence, (0,60), cv2.FONT_HERSHEY_SIMPLEX, 0.8,color, 2, cv2.LINE_AA)
    return label_Voilence

In [15]:
def avg_predictions():
    global weapon_array
    global mask_array
    global Shoplifting
    global Voilence
        
    global features_shoplifting 
    global features_Voilence 

    global count_shoplifting 
    global count_Voilence 
    
    global AVG_Mask_Decision 
    global AVG_Weapon_Decision
#-----------------------------------------------------------------
    count_shoplifting = count_shoplifting +1
    count_Voilence = count_Voilence +1
    #----------------------------------------
    if count_shoplifting > 30 :
        features_shoplifting = []
        count_shoplifting  = 0
#AVG_Weapn -------------------------------------------------------------------------------------------
    if count_Voilence < 40 :
        if Weapon[0:6] == "Weapon":

            weapon_array.append(1)
        else: 
            weapon_array.append(0)
    else:
        print(weapon_array)
        Weapon_Avg = np.mean(weapon_array)
        print("Weapon_Avg >> " + str(Weapon_Avg))
        if Weapon_Avg >= 0.35:
            AVG_Weapon_Decision = "Weapon"
        else:
            AVG_Weapon_Decision = "No Weapon"
        print("Weapon Array :: ")
        weapon_array =[]
        print("----------------------------------------------------")

#AVG_Mask -------------------------------------------------------------------------------------------
    if count_Voilence < 40 :
        if  Mask == "Threat":
            mask_array.append(1)
        else:
            mask_array.append(0)
    else:
        print(mask_array)
        Mask_Avg = np.mean(mask_array)
        print("Mask Avg :: "+str(Mask_Avg))
        if Mask_Avg >= 0.35:
            print("Threat")
            AVG_Mask_Decision = "Threat"
        else: 
            print("No Threat")
            AVG_Mask_Decision = "No Threat"
            
        print("Mask_Array :: ")
        mask_array =[]
        print("----------------------------------------------------")

In [16]:
# model_Shoplifting = tf.keras.models.load_model(r"C:\Users\mnaga\Desktop\Shplifting_live\shoplifting_30frames_CONVLSTM3_25.h5")

In [17]:
pwd

'C:\\Users\\mnaga\\Desktop\\Graduation_Project\\Custom_Weapon'

In [18]:
cd ..

C:\Users\mnaga\Desktop\Graduation_Project


In [19]:
# Mask ::: 
mask_dict = {0: "Masked-face", 1: "Mask", 2: "Without_Mask"}
prototxtPath = r"H5_Models\haarcascade_xml\deploy.prototxt"
weightsPath = r"H5_Models\haarcascade_xml\res10_300x300_ssd_iter_140000.caffemodel"
maskNet = tf.keras.models.load_model(r"Mask_Model\Mask_model.h5")
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)
#---------------------------------
#Shoplifting ::
model_Shoplifting = tf.keras.models.load_model(r"Shplifting_live\shoplifting_30frames_CONVLSTM3_25.h5")
#------------------------------------------
vid_model = VGG16(include_top=True, weights='imagenet')
transfer_layer = vid_model.get_layer('fc2')
image_model_transfer = Model(inputs=vid_model.input,outputs=transfer_layer.output)
model_Voilence = tf.keras.models.load_model(r"Voilence_Model\Violencev5.h5")
img_size = 224
#------------------------------------------

In [20]:
# facial_model = tf.keras.models.load_model(r"H5_Models\Facial_Model_v6.h5")

In [21]:
# def fecial (frame): 
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     face_cascade = cv2.CascadeClassifier(r"C:\Users\mnaga\Desktop\H5_Models\haarcascade_xml\haarcascade_frontalface_default.xml")
#     faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
#     for (x, y, w, h) in faces:
#         cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 1)
#         roi_gray = gray[y:y + h, x:x + w]
#         cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
#         cv2.normalize(cropped_img, cropped_img, alpha=0, beta=1, norm_type=cv2.NORM_L2, dtype=cv2.CV_32F)
#         prediction = facial_model.predict(cropped_img)
#         cv2.putText(frame, emotion_dict[int(np.argmax(prediction))], (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1, cv2.LINE_AA)
#         return emotion_dict[int(np.argmax(prediction))]

In [22]:
from twilio.rest import Client

In [23]:
def SMS_Alert(case):
    account_sid = "AC7c9122d3a1cedc051d1e914d56ed4b68"
    auth_token = "b1791e6e215d22117de5aac6a774c853"
    client = Client(account_sid , auth_token)
    client.messages.create(from_="+12019421776" , body=case ,to="+201275599845")
    print("SMS >> sended")

In [24]:
def final_Decision ():
    global AVG_Mask_Decision 
    global AVG_Weapon_Decision
    
    global Shoplifting
    global Voilence
    
    if Shoplifting == "Shoplifting" :
        #SMS_Alert("Alert : "+Shoplifting)
        return
    elif AVG_Weapon_Decision == "Weapon" :
        #SMS_Alert("Alert : "+AVG_Weapon_Decision)
        return
    
    elif Voilence == "Violence":
        #SMS_Alert("Alert : "+ Voilence)
        return
    
    elif AVG_Mask_Decision  == "Threat" :
        #SMS_Alert("Alert : "+ AVG_Mask_Decision )
        return
    else : 
        return

In [25]:
# cd C:\Users\mnaga\Desktop\Notebook's

In [ ]:
#open webcam .. 
# initialize the video stream
print("[INFO] starting video stream...")

label_shoplifting = "shoplifting >> neutral"
label_Voilence = "violence >> neutral"

features_shoplifting = []
features_Voilence = []

count_shoplifting = 0 
count_Voilence = 0

weapon_array = []
mask_array = [] 

AVG_Mask_Decision ="No Avg decision for Mask"
AVG_Weapon_Decision ="No Avg decision for Weapon"

cap = cv2.VideoCapture(1)
#cap = cv2.VideoCapture("shoplifting.mp4")
while True:
    ret,frame = cap.read()
    if ret == False : 
        break
    frame = imutils.resize(frame, width=700)
    color = (0, 0, 255)
    #----------------------------------------
    #Mask ::
    Mask = mask(frame, faceNet, maskNet)
    #Weapon ::
    Weapon , frame = weapon(frame)
    # Shoplifting ::
    Shoplifting =Shoplifting_prediction(frame,features_shoplifting)
    #Voilence :: 
    Voilence = Voilence_prediction (frame,features_Voilence)
    #-------------
    avg_predictions()
    #-------------
    if count_Voilence == 40 :
        final_Decision ()
        features_Voilence = []
        count_Voilence = 0
    #--------------------
    cv2.imshow("frame" , frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
#--------------------------------------------------      
frame =[]
features_shoplifting = []
features_Voilence = []
count_shoplifting = 0 
count_Voilence = 0
weapon_array = []
mask_array = [] 
#-------------------------------------------------
cap.release()
cv2.destroyAllWindows()

[INFO] starting video stream...
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
Normal with : 0.9998134
(1, 1, 200, 7)
(320, 235, 3)
